Primero se cargan las librerias que se usarán:

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor as mlp_r
from sklearn.neural_network import MLPClassifier as mlp_c
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import datasets


## Regresión

El ejercicio de regresión se realizará con la base de datos  ["Wine Quality"](https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009). Es fundamental antes de comenzar cualquier ejercicio de Machine Learning o Inteligencia Artificial conocer los datos, por lo cual se les recomienda realizar un Análisis Exploratorio de los Datos previamente ([EDA](https://towardsdatascience.com/a-gentle-introduction-to-exploratory-data-analysis-f11d843b8184)). Primero cargamos la base de datos de vinos:

In [ ]:
wine_data = pd.read_csv("../input/winequality-red.csv")
wine_data.head()

Se construyen nuestras variables de trabajo, por lo notaciòn "X" corresponde a las variables independientes y "y" a la variable dependiente.

In [ ]:
X = (wine_data.iloc[:,:-1]).as_matrix()
y = (wine_data.iloc[:,-1]).as_matrix()


Llevamos nuestras variables al hypercubo (0,1). **NOTA**: La variable dependiente se divide por 10 dado que se conoce la naturaleza de los datos.

In [ ]:
X = (X - X.min(axis=0))/(X.max(axis=0) - X.min(axis=0))
y = y.reshape((-1,1))/10


Se divide nuestro dataset en dos subconjuntos: Train y Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

Inicializamos nuestro MLP y entrenamos:

In [ ]:
regressor = mlp_r(
    hidden_layer_sizes=(100,),  activation='tanh', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
y_hats = regressor.predict(X_test)


Contrastamos el resultado de nuestro estimador con el estimador ideal:

In [ ]:
plt.scatter(y_test, y_hats, c='k')

plt.plot([0.2, 0.9], [0.2, 0.9], 'r')
plt.xlabel('Real')
plt.ylabel('Estimada')

Realizamos el  mismo procedimiento reduciendo las dimensiones de los datos:

In [ ]:
pca = PCA(n_components=3)

X_pca = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, np.ravel(y), test_size=0.33, random_state=42)

regressor2 = mlp_r(
    hidden_layer_sizes=(100,),  activation='tanh', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=0, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

regressor2.fit(X_train, y_train)

y_hat2 = regressor2.predict(X_test)

In [ ]:
plt.scatter(y_test, y_hat2, c='k')

plt.plot([0.2, 0.8], [0.2, 0.8], 'r')
plt.xlabel('Real')
plt.ylabel('Estimada')


**TO DO**
1. Como sabemos que el modelo esta bien entrenado y como sabemos si es un buen modelo?  [Aplicar métricas de evaluación](https://scikit-learn.org/stable/model_selection.html#model-selection)
2. Seleccionar 1 métrica de evaluación (mse, mape, mae, r2, F-score, AIC, etc.), y realizar un grafico de este bi-dimensional del promedio de dicha métrica y otro de la desviación estandár de dicha métrica, en el cual se varie la cantidad de neuronas en la primer capa, y en la segunda capa.
3. Repetir el experimento anterior seleccionando el mejor modelo, pero variando el momentum (por lo tanto es un grafico unidimensional).

**TO DO 1**

Se selecciona la métrica [mse](https://en.wikipedia.org/wiki/Mean_squared_error) (clasica para evaluar ejercicios de regresión), y se realiza el gráfico bi-dimensional:

In [ ]:
from sklearn.metrics import mean_squared_error

capa_1 = [5, 7, 9, 11, 13, 17,  19, 23, 29, 31]
capa_2 = [1, 5, 7, 9, 11, 13, 17,  19, 23, 29]

mse_m = np.zeros((len(capa_1),len(capa_1)))
mse_std = np.zeros((len(capa_1),len(capa_1)))

for j, n_1 in enumerate(capa_1):
    for k, n_2 in enumerate(capa_2):
        mse_temp = []
    
        for i in range(10):
            X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.33)
            
            regresor_temp = mlp_r(hidden_layer_sizes=(n_1, n_2, ),  activation='tanh')
            regresor_temp.fit(X_train, y_train)
            y_pred = regressor2.predict(X_test)
            mse_temp.append(mean_squared_error(y_test, y_pred))
        mse_m[j, k] = np.mean(mse_temp)
        mse_std[j, k] = np.std(mse_temp)


In [ ]:
plt.imshow(mse_m)
plt.colorbar()

In [ ]:
plt.imshow(mse_std)
plt.colorbar()

## Clasificación

El ejercicio de clasificación se hará empleando la famosa base de datos de Iris. Primero cargamos los datos:

In [ ]:
iris = datasets.load_iris()
X = iris.data 
y = iris.target

Generamos los conjuntos de entrenamiento y prueba:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)


Inicializamos nuestro MLP y entrenamos:

In [ ]:
clf = mlp_c(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(100, 2), random_state=1)

clf.fit(X_train, y_train)     

In [ ]:
y_hat = clf.predict(X_test)

Se calcula la matriz de confusión para el clasificador:

In [ ]:
cm1 = confusion_matrix(y_test, y_hat)
cm2 = confusion_matrix(y_test, y_hat)
cm2 = cm2.astype('float') / cm2.sum(axis=1)[:, np.newaxis]



Sin normalizar

In [ ]:
cm1

Normalizada

In [ ]:
cm2

**TO DO**
1. Como sabemos que el modelo esta bien entrenado y como sabemos si es un buen modelo?  [Aplicar métricas de evaluación](https://scikit-learn.org/stable/model_selection.html#model-selection)
2. Seleccionar un dataset nuevo, implementar validación cruzada con k-folds, y encontrar los parametros con los cuales se puede lograr clasificar el dataset escogido.


# Tarea:

1. Implementar un algoritmo de clustering de redes neuronales, tomar de referencia: https://hagan.okstate.edu/NNDesign.pdf 
2. Aplicar este algoritmo a una base de datos de Kaggle, y aplicar métricas de evaluación como:
    * Adjusted Rand index
    * Mutual Information
    * Silhouette Coefficient
    * Calinski-Harabasz Index
    * Entre otros.
2. TO DO de Regresión
3. Conclusiones sobre los resultados obtenidos
4. TO DO de Clasificación
5. Conclusiones sobre los resultados obtenidos